# Load Dataset

In [1]:
from datasets import load_dataset, load_from_disk
import json 
import pandas as pd
import re

/opt/conda/envs/mrc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## train/test dataset

In [2]:
dataset = load_from_disk('../../../data/train_dataset')
train_dataset = dataset['train']
val_dataset = dataset['validation']

In [3]:
test_dataset = load_from_disk('../../../data/test_dataset')
test_dataset = test_dataset['validation']

## Wiki dataset

In [4]:
wiki_path = '../../../data/wikipedia_documents.json'
with open(wiki_path, 'r',encoding='utf-8') as f:
    wiki = json.load(f)

In [5]:
wiki_df = pd.DataFrame(wiki).T
wiki_text = list(wiki_df.text)

In [6]:
def find_special_characters(data):
    pattern = r'[^\w\sㄱ-ㅎㅏ-ㅣ가-힣]'  # 영숫자, 공백, 한글이 아닌 모든 문자를 찾는 패턴
    special_chars = re.findall(pattern, data)
    # return special_chars
    return list(set(special_chars))

In [7]:
wiki_doc = ' '.join(wiki_text)
print(len(wiki_doc))
print(sum([len(text) for text in wiki_text]) + len(wiki_text) -1)

45857676
45857676


In [8]:
special_character = find_special_characters(wiki_doc)

# 특수문자 확인

In [9]:
total_train_df = pd.DataFrame(train_dataset)
total_train_df = pd.concat([total_train_df, pd.DataFrame(val_dataset)]).reset_index(drop=True)
total_train_df.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [10]:
total_train_df["answer_txt"] = total_train_df['answers'].apply(lambda x: x['text'][0])
total_train_df = total_train_df[["title", "context", "question", "answer_txt", "document_id"]]
total_train_df.head()

,title,context,question,answer_txt,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,4개,47334


In [11]:
answer_txt = list(total_train_df['answer_txt'])

special_tok = special_character.copy() # wiki에서 확인 가능했던 특수문자들

in_answer =[]
print("show answer example")
for i, word in enumerate(answer_txt):
    for p in special_tok:
        if p in word:
            print("\t",i, word, p)
            in_answer.append(p)
            special_tok.remove(p)

print()
print()
print('total special char :', special_character) # '\ue6ac'형태로 출력되는 애는 unicode 특수문자가 string type으로 저장될 때 '\ue6ac'형태로 저장된 것 
print('special char shown in answer:', in_answer)
print('special char not in  answer:', special_tok)



show answer example
	 1 《경영의 실제》 《
	 1 《경영의 실제》 》
	 15 '초일기' '
	 30 우전왕(優塡王)식 착의법 (
	 30 우전왕(優塡王)식 착의법 )
	 58 ≪인간의 이해: 개념의 집단적 사용 및 진화 (1972)≫ :
	 58 ≪인간의 이해: 개념의 집단적 사용 및 진화 (1972)≫ ≪
	 58 ≪인간의 이해: 개념의 집단적 사용 및 진화 (1972)≫ ≫
	 62 〈마브루공(公)〉 〉
	 62 〈마브루공(公)〉 〈
	 136 ‘전의식’(前意識) ’
	 136 ‘전의식’(前意識) ‘
	 146 연평균 10% %
	 295 요한 B. 크라머 .
	 366 1,331개 ,
	 520 "오 그 불쌍한 사람들... 그러나 그건 별로 중요하지 않아요! 중요한건 난 새로운 배달원이 필요했다는 거지!" !
	 520 "오 그 불쌍한 사람들... 그러나 그건 별로 중요하지 않아요! 중요한건 난 새로운 배달원이 필요했다는 거지!" "
	 544 최근의 반도의 경제사범 ­국민의 신경제 윤리의 파악을 위하여­ ­
	 557 5-8월 -
	 745 “자랑스런 죄수” “
	 745 “자랑스런 죄수” ”
	 812 네우스트리아·부르고뉴·프로방스 ·
	 920 <유추론자의 이론에 대하여> <
	 920 <유추론자의 이론에 대하여> >
	 1030 ‘∧’ 모양 ∧
	 1421 『전체주의의 기원』 『
	 1421 『전체주의의 기원』 』
	 1980 「남행월일기」 「
	 1980 「남행월일기」 」
	 2041 ＜펠리사＞ ＞
	 2041 ＜펠리사＞ ＜
	 2324 오카야마・아노쓰 번 ・
	 2501 80 km/h /
	 2807 2011년~2013년 ~
	 3280 27 ° C °
	 3782 펄사 PSR J0205+64 +


total special char : ['\uf34b', '】', '`', '\ue284', '\uf53d', 'ླ', '⇕', '\ue911', '）', '\ue0cd', '\ue1bf', '《', 'ྷ', '\ue6e9',

In [12]:
text = ""
for sp in in_answer:
    text += sp
print(text)

《》'():≪≫〉〈’‘%.,!"­-“”·<>∧『』「」＞＜・/~°+


In [13]:
def preprocess(text: str) -> str:
    text = re.sub(r"\\+n", " ", text)
    text = re.sub(r"#", " ", text)
    text = re.sub(
        # r"([^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z\d\s\.\,\'\"\<\>\!\@\#\$\%\^\&\*\(\)\[\]\_\+\-《》)(≫≪:〈〉‘’%.,!­-”“·><∧』『」「＜＞・/~°+])", # reader
        r"([^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z\d\s\.\,\'\"\<\>\!\@\#\$\%\^\&\*\(\)\[\]\_\+\-’‘“”《》〈〉~])", # retriever
        "",
        text,
    )
    text = re.sub(r"\s+", " ", text).strip()  # 중복 공백 제거

    return text


In [14]:
total_train_df['context'] = total_train_df['context'].apply(preprocess)